In [1]:
import tensorflow as tf
import numpy as np
import PIL
import random

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
print("Done")

Done


In [2]:
random.seed(0)
np.random.seed(0)
tf.random.set_seed(42)
tf.random.set_seed(42)

In [3]:
BATCH_SIZE = 32
IMG_SIZE = (160,160)

train_ds = image_dataset_from_directory(
    "../animals", # Change according to the folder containing the dataset
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = image_dataset_from_directory(
    "../animals", # Change according to the folder containing the dataset
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches//5)
val_ds = val_ds.skip(val_batches//5)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(160,160,3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
])

Found 3000 files belonging to 3 classes.
Using 2400 files for training.
Found 3000 files belonging to 3 classes.
Using 600 files for validation.


In [4]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

IMG_SHAPE = IMG_SIZE+(3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights="imagenet")
image_batch,label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)

base_model.trainable = False
base_model.summary()

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

prediction_layer = tf.keras.layers.Dense(3,activation="softmax")
prediction_batch = prediction_layer(feature_batch_average)

inputs = tf.keras.Input(shape=(160,160,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x,training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs,outputs)

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()

initial_epochs = 15
loss0,accuracy0 = model.evaluate(val_ds)
print("Initial loss: {:.2f} %".format(100*loss0))
print("Initial accuracy: {:.2f} %".format(100*accuracy0))
checkpoint = ModelCheckpoint("mobilenetv2.hdf5",monitor="val_accuracy",save_best_only=True,save_weights_only=False)
model.fit(train_ds,epochs=initial_epochs,validation_data=val_ds,callbacks=[checkpoint])
best = tf.keras.models.load_model("mobilenetv2.hdf5")
loss,accuracy = best.evaluate(test_ds)
print("\nTest acc: {:.2f} %".format(100*accuracy))
print("Test loss: {:.2f} %".format(100*loss))

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0     

/home/maverick/.local/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/15
75/75 [==============================] - 4s 55ms/step - loss: 0.1204 - accuracy: 0.9571 - val_loss: 0.0466 - val_accuracy: 0.9841
Epoch 3/15
75/75 [==============================] - 4s 54ms/step - loss: 0.0913 - accuracy: 0.9638 - val_loss: 0.0645 - val_accuracy: 0.9762
Epoch 4/15
75/75 [==============================] - 4s 54ms/step - loss: 0.0821 - accuracy: 0.9712 - val_loss: 0.0456 - val_accuracy: 0.9841
Epoch 5/15
75/75 [==============================] - 4s 53ms/step - loss: 0.0666 - accuracy: 0.9754 - val_loss: 0.0371 - val_accuracy: 0.9881
Epoch 6/15
75/75 [==============================] - 4s 57ms/step - loss: 0.0623 - accuracy: 0.9762 - val_loss: 0.0300 - val_accuracy: 0.9921
Epoch 7/15
75/75 [==============================] - 4s 56ms/step - loss: 0.0633 - accuracy: 0.9771 - val_loss: 0.0338 - val_accuracy: 0.9901
Epoch 8/15
75/75 [==============================] - 4s 55ms/step - loss: 0.0559 - accuracy: 0.9812 - val_loss: 0.0480 - val_accuracy: 0.9762
Epoch 9/15
75